In [1]:
import glob
import os
import zipfile
import pandas as pd

In [2]:
paths = list(filter(lambda path: 'T_ONTIME_REPORTING' in path, glob.glob('/Users/leonhards/Downloads/*.zip')))

In [3]:
len(paths)

1143

In [4]:
test='319906743_T_ONTIME_REPORTING.zip'

In [5]:
#paths = [ path for path in paths if test in path]

In [6]:
#path

In [7]:
# check what files are contained
path = paths[0]
with zipfile.ZipFile(path, 'r') as zf:
    print(zf.namelist())

['106108242_T_ONTIME_REPORTING.csv']


In [8]:
zf = zipfile.ZipFile(path)

zf.infolist()

df_snippet = pd.read_csv(zf.open(zf.namelist()[0]), nrows=10, encoding = "ISO-8859-1")

In [9]:
df_snippet

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM,Unnamed: 109
0,2007,2,5,30,3,2007-05-30,AA,19805,AA,N3AUAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007,2,5,31,4,2007-05-31,AA,19805,AA,N3BHAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007,2,5,1,2,2007-05-01,AA,19805,AA,N494AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007,2,5,2,3,2007-05-02,AA,19805,AA,N4WXAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007,2,5,3,4,2007-05-03,AA,19805,AA,N592AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2007,2,5,4,5,2007-05-04,AA,19805,AA,N4WBAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2007,2,5,5,6,2007-05-05,AA,19805,AA,N414AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2007,2,5,6,7,2007-05-06,AA,19805,AA,N531AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2007,2,5,7,1,2007-05-07,AA,19805,AA,N4XHAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2007,2,5,8,2,2007-05-08,AA,19805,AA,N4XTAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import pandas as pd
from tqdm import tqdm

In [11]:
lookup_dict = {}

for path in tqdm(paths):
    try:
        with zipfile.ZipFile(path) as z:
            #print(z.infolist())
            #print(z.infolist()[0].filename)
            
            # find the right filename
            infos = z.infolist()
            ontime_name = ''
            for info in infos:
                if not 'Readme' in info.filename and not 'Terms' in info.filename:
                    ontime_name = info.filename
                    break
            #print(info)
            df_snippet = pd.read_csv(z.open(ontime_name), nrows=10, encoding = "ISO-8859-1")
            year, month = tuple(df_snippet.iloc[0][['YEAR', 'MONTH']].values)
            lookup_dict[path] = (year, month)
            #print('Found: {}/{}'.format(year, month))
    except Exception as e:
        #print(e)
        pass

100%|██████████| 1143/1143 [00:12<00:00, 91.03it/s]


In [12]:
S = list(lookup_dict.values())
S[:2]

[(2007, 5), (2005, 5)]

In [13]:

missing_list = []
for year in range(1988, 2022):
    for month in range(1, 13):
        if not (year, month) in S:
            #print('Missing: {}/{}'.format(year, month))
            
            missing_list.append((year, month))

In [14]:
len(missing_list)

1

In [15]:
len(missing_list)

1

In [16]:
missing_list

[(2021, 12)]

In [26]:
def get_zip_year_month(path):
    try:
        with zipfile.ZipFile(path) as z:
            # find the right filename
            infos = z.infolist()
            ontime_name = ''
            for info in infos:
                if not 'Readme' in info.filename and not 'Terms' in info.filename:
                    ontime_name = info.filename
                    break
            #print(info)
            df_snippet = pd.read_csv(z.open(ontime_name), nrows=10, encoding = "ISO-8859-1")
            year, month = tuple(df_snippet.iloc[0][['YEAR', 'MONTH']].values)
            return year, month
    except Exception as e:
        return None, None

In [46]:
# Function : file_compress
# from https://www.tutorialspoint.com/how-to-compress-files-with-zipfile-module-in-python
def file_compress(inp_file_names, out_zip_file):
    """
    function : file_compress
    args : inp_file_names : list of filenames to be zipped
    out_zip_file : output zip file
    return : none
    assumption : Input file paths and this code is in same directory.
    """
    # Select the compression mode ZIP_DEFLATED for compression
    # or zipfile.ZIP_STORED to just store the file
    compression = zipfile.ZIP_DEFLATED
    print(f" *** Input File name passed for zipping - {inp_file_names}")

    # create the zip file first parameter path/name, second mode
    print(f' *** out_zip_file is - {out_zip_file}')
    zf = zipfile.ZipFile(out_zip_file, mode="w")

    try:
        for file_to_write in inp_file_names:
            # Add file to the zip file
            # first parameter file to zip, second filename in zip
            print(f' *** Processing file {file_to_write}')
            zf.write(file_to_write, os.path.basename(file_to_write), compress_type=compression)

    except FileNotFoundError as e:
        print(f' *** Exception occurred during zip process - {e}')
    finally:
        # Don't forget to close the file!
        zf.close()

In [ ]:
# Function : file_compress
# from https://www.tutorialspoint.com/how-to-compress-files-with-zipfile-module-in-python
def gzip_file(inp_file_names, out_zip_file):
    """
    function : file_compress
    args : inp_file_names : list of filenames to be zipped
    out_zip_file : output zip file
    return : none
    assumption : Input file paths and this code is in same directory.
    """
    # Select the compression mode ZIP_DEFLATED for compression
    # or zipfile.ZIP_STORED to just store the file
    compression = zipfile.ZIP_DEFLATED
    print(f" *** Input File name passed for zipping - {inp_file_names}")

    # create the zip file first parameter path/name, second mode
    print(f' *** out_zip_file is - {out_zip_file}')
    zf = zipfile.ZipFile(out_zip_file, mode="w")

    try:
        for file_to_write in inp_file_names:
            # Add file to the zip file
            # first parameter file to zip, second filename in zip
            print(f' *** Processing file {file_to_write}')
            zf.write(file_to_write, os.path.basename(file_to_write), compress_type=compression)

    except FileNotFoundError as e:
        print(f' *** Exception occurred during zip process - {e}')
    finally:
        # Don't forget to close the file!
        zf.close()

In [76]:
import shutil
import gzip
import shutil

In [ ]:
# reorganize raw dataset, i.e. the zipfiles! => write and compress them then individually!
lookup_dict


dest_dir = 'full_flight_data'
os.makedirs(dest_dir, exist_ok=True)
name_template = 'flights_on_time_performance_{:04d}_{:02d}.csv'

processed = {}

count = 0
for k, v in list(lookup_dict.items()):
    fsize = os.stat(k).st_size
    
    # extract year, month from file
    zpath = k
    year, month = get_zip_year_month(zpath)
    
    k = (year, month)
    if k in processed.keys():
        # check if larger, if so use that file!
        if fsize < processed[k]:
            # use current file
            path = None
            continue
    # path
    if year and month:
        
        # extract/copy file
        try:
            with zipfile.ZipFile(zpath) as z:
                # find the right filename
                infos = z.infolist()
                ontime_name = ''
                for info in infos:
                    if not 'Readme' in info.filename and not 'Terms' in info.filename:
                        ontime_name = info.filename
                        break
                # year, month known from above. Time to extract file!
                name = name_template.format(year, month)
                dest_path = os.path.join(dest_dir, name_template.format(year, month))
                workdir = os.path.join(dest_dir, 'work')
                z.extract(ontime_name, workdir)
                csv_files = glob.glob(workdir + "/*.csv")
                print('Extracted {}/{} {}...'.format(count, len(lookup_dict), name))
                
                # rename the first csv files
                if len(csv_files) != 1:
                    print('ERROR: {}'.format(csv_files))
                    continue
                csvfile_path = csv_files[0]
                os.rename(csvfile_path, dest_path)
                
                ## compress file 
                #file_compress([dest_path], dest_path + ".gz")
                with open(dest_path, 'rb') as f_in:
                    with gzip.open(dest_path + '.gz', 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                
                shutil.rmtree(workdir)
                os.remove(dest_path)
        except Exception as e:
            print('failed for {}, {}'.format(zpath, e))
        
        processed[k] = fsize
        count += 1

Extracted 0/1129 flights_on_time_performance_2007_05.csv...
Extracted 1/1129 flights_on_time_performance_2005_05.csv...
Extracted 2/1129 flights_on_time_performance_2005_12.csv...
Extracted 3/1129 flights_on_time_performance_1999_06.csv...
Extracted 4/1129 flights_on_time_performance_2018_06.csv...
Extracted 5/1129 flights_on_time_performance_2006_09.csv...
Extracted 6/1129 flights_on_time_performance_1992_09.csv...
Extracted 7/1129 flights_on_time_performance_2006_09.csv...
Extracted 8/1129 flights_on_time_performance_2020_03.csv...
Extracted 9/1129 flights_on_time_performance_2005_12.csv...
Extracted 10/1129 flights_on_time_performance_2004_02.csv...
Extracted 11/1129 flights_on_time_performance_2007_05.csv...
Extracted 12/1129 flights_on_time_performance_2006_09.csv...
Extracted 13/1129 flights_on_time_performance_1997_05.csv...
Extracted 14/1129 flights_on_time_performance_2006_09.csv...
Extracted 15/1129 flights_on_time_performance_2007_05.csv...
Extracted 16/1129 flights_on_time_

Extracted 134/1129 flights_on_time_performance_2005_12.csv...
Extracted 135/1129 flights_on_time_performance_2016_08.csv...
Extracted 136/1129 flights_on_time_performance_1989_10.csv...
Extracted 137/1129 flights_on_time_performance_1998_07.csv...
Extracted 138/1129 flights_on_time_performance_2004_02.csv...
Extracted 139/1129 flights_on_time_performance_1998_07.csv...
Extracted 140/1129 flights_on_time_performance_2008_11.csv...
Extracted 141/1129 flights_on_time_performance_2000_07.csv...
Extracted 142/1129 flights_on_time_performance_2016_08.csv...
Extracted 143/1129 flights_on_time_performance_1989_09.csv...
Extracted 144/1129 flights_on_time_performance_2001_05.csv...
Extracted 145/1129 flights_on_time_performance_2007_05.csv...
Extracted 146/1129 flights_on_time_performance_2005_12.csv...
Extracted 147/1129 flights_on_time_performance_2006_09.csv...
Extracted 148/1129 flights_on_time_performance_1994_06.csv...
Extracted 149/1129 flights_on_time_performance_2017_08.csv...
Extracte

Extracted 267/1129 flights_on_time_performance_1997_06.csv...
Extracted 268/1129 flights_on_time_performance_2006_09.csv...
Extracted 269/1129 flights_on_time_performance_1992_10.csv...
Extracted 270/1129 flights_on_time_performance_2006_09.csv...
Extracted 271/1129 flights_on_time_performance_2020_03.csv...
Extracted 272/1129 flights_on_time_performance_2007_05.csv...
Extracted 273/1129 flights_on_time_performance_2005_05.csv...
Extracted 274/1129 flights_on_time_performance_2005_12.csv...
Extracted 275/1129 flights_on_time_performance_1999_06.csv...
Extracted 276/1129 flights_on_time_performance_2006_05.csv...
Extracted 277/1129 flights_on_time_performance_2006_09.csv...
Extracted 278/1129 flights_on_time_performance_2007_05.csv...
Extracted 279/1129 flights_on_time_performance_2006_09.csv...
Extracted 280/1129 flights_on_time_performance_2006_09.csv...
Extracted 281/1129 flights_on_time_performance_2020_03.csv...
Extracted 282/1129 flights_on_time_performance_2015_03.csv...
Extracte

Extracted 400/1129 flights_on_time_performance_2003_01.csv...
Extracted 401/1129 flights_on_time_performance_2007_05.csv...
Extracted 402/1129 flights_on_time_performance_2006_09.csv...
Extracted 403/1129 flights_on_time_performance_2018_12.csv...
Extracted 404/1129 flights_on_time_performance_2004_06.csv...
Extracted 405/1129 flights_on_time_performance_2003_05.csv...
Extracted 406/1129 flights_on_time_performance_2012_04.csv...
Extracted 407/1129 flights_on_time_performance_1995_10.csv...
Extracted 408/1129 flights_on_time_performance_2006_09.csv...
Extracted 409/1129 flights_on_time_performance_1998_05.csv...
Extracted 410/1129 flights_on_time_performance_1999_06.csv...
Extracted 411/1129 flights_on_time_performance_2004_02.csv...
Extracted 412/1129 flights_on_time_performance_2015_03.csv...
Extracted 413/1129 flights_on_time_performance_2006_09.csv...
Extracted 414/1129 flights_on_time_performance_2007_05.csv...
Extracted 415/1129 flights_on_time_performance_1988_05.csv...
Extracte

Extracted 533/1129 flights_on_time_performance_2006_09.csv...
Extracted 534/1129 flights_on_time_performance_2016_08.csv...
Extracted 535/1129 flights_on_time_performance_2007_05.csv...
Extracted 536/1129 flights_on_time_performance_2007_05.csv...
Extracted 537/1129 flights_on_time_performance_2006_09.csv...
Extracted 538/1129 flights_on_time_performance_2016_04.csv...
Extracted 539/1129 flights_on_time_performance_2005_12.csv...
Extracted 540/1129 flights_on_time_performance_2005_12.csv...
Extracted 541/1129 flights_on_time_performance_2007_05.csv...
Extracted 542/1129 flights_on_time_performance_2000_04.csv...
Extracted 543/1129 flights_on_time_performance_1997_12.csv...
Extracted 544/1129 flights_on_time_performance_2006_09.csv...
Extracted 545/1129 flights_on_time_performance_2009_03.csv...
Extracted 546/1129 flights_on_time_performance_2007_07.csv...
Extracted 547/1129 flights_on_time_performance_2013_05.csv...
Extracted 548/1129 flights_on_time_performance_2006_09.csv...
Extracte

Extracted 666/1129 flights_on_time_performance_1989_02.csv...
Extracted 667/1129 flights_on_time_performance_2007_05.csv...
Extracted 668/1129 flights_on_time_performance_2016_08.csv...
Extracted 669/1129 flights_on_time_performance_2016_03.csv...
Extracted 670/1129 flights_on_time_performance_2010_09.csv...
Extracted 671/1129 flights_on_time_performance_2007_05.csv...
Extracted 672/1129 flights_on_time_performance_2006_09.csv...
Extracted 673/1129 flights_on_time_performance_2006_09.csv...
Extracted 674/1129 flights_on_time_performance_1996_05.csv...
Extracted 675/1129 flights_on_time_performance_2007_05.csv...
Extracted 676/1129 flights_on_time_performance_2004_02.csv...
Extracted 677/1129 flights_on_time_performance_2014_08.csv...
Extracted 678/1129 flights_on_time_performance_2013_05.csv...
Extracted 679/1129 flights_on_time_performance_2006_09.csv...
Extracted 680/1129 flights_on_time_performance_2007_05.csv...
Extracted 681/1129 flights_on_time_performance_1997_09.csv...
Extracte

Extracted 799/1129 flights_on_time_performance_2012_09.csv...
Extracted 800/1129 flights_on_time_performance_2006_09.csv...
Extracted 801/1129 flights_on_time_performance_1987_10.csv...
Extracted 802/1129 flights_on_time_performance_2005_12.csv...
Extracted 803/1129 flights_on_time_performance_2004_01.csv...
Extracted 804/1129 flights_on_time_performance_2008_09.csv...
Extracted 805/1129 flights_on_time_performance_2005_01.csv...
Extracted 806/1129 flights_on_time_performance_1990_09.csv...
Extracted 807/1129 flights_on_time_performance_2007_05.csv...
Extracted 808/1129 flights_on_time_performance_2016_08.csv...
Extracted 809/1129 flights_on_time_performance_2014_03.csv...
Extracted 810/1129 flights_on_time_performance_2005_10.csv...
Extracted 811/1129 flights_on_time_performance_1997_12.csv...
Extracted 812/1129 flights_on_time_performance_2016_09.csv...
Extracted 813/1129 flights_on_time_performance_1993_06.csv...
Extracted 814/1129 flights_on_time_performance_2005_12.csv...
Extracte

Extracted 932/1129 flights_on_time_performance_2007_05.csv...
Extracted 933/1129 flights_on_time_performance_1993_03.csv...
Extracted 934/1129 flights_on_time_performance_2017_02.csv...
Extracted 935/1129 flights_on_time_performance_1998_07.csv...
Extracted 936/1129 flights_on_time_performance_2005_06.csv...
Extracted 937/1129 flights_on_time_performance_2013_11.csv...
Extracted 938/1129 flights_on_time_performance_2013_09.csv...
Extracted 939/1129 flights_on_time_performance_1990_12.csv...
Extracted 940/1129 flights_on_time_performance_2016_08.csv...
Extracted 941/1129 flights_on_time_performance_2004_01.csv...
Extracted 942/1129 flights_on_time_performance_2005_12.csv...
Extracted 943/1129 flights_on_time_performance_2006_09.csv...
Extracted 944/1129 flights_on_time_performance_2007_05.csv...
Extracted 945/1129 flights_on_time_performance_2006_09.csv...
Extracted 946/1129 flights_on_time_performance_2012_12.csv...
Extracted 947/1129 flights_on_time_performance_2021_06.csv...
Extracte

In [49]:
print('done!')

done!


In [ ]:
shutil.rmtree()

In [30]:
len(processed)

410

In [50]:
# make sure data is complete
paths = glob.glob('full_flight_data/*.csv.gz')

In [57]:
names = list(map(os.path.basename, sorted(paths)))

In [58]:
import re

In [75]:
S = set()
for name in names:
    m = re.search('flights_on_time_performance_(\d\d\d\d)_(\d\d).csv.gz', name)
    year, month = int(m[1]), int(m[2])

    # check files correspond to name
    df_snippet = pd.read_csv('full_flight_data/' + name, nrows=10, encoding = "ISO-8859-1")
    dyear, dmonth = tuple(df_snippet.iloc[0][['YEAR', 'MONTH']].values)
    if dyear != year or dmonth != month:
        print(dyear, year, dmonth, month)
    S.add((year, month))

BadGzipFile: Not a gzipped file (b'PK')

In [73]:

missing_list = []
for year in range(1987, 2022):
    for month in range(1, 13):
        if not (year, month) in S:
            #print('Missing: {}/{}'.format(year, month))
            
            missing_list.append((year, month))

In [74]:
missing_list

[(1987, 1),
 (1987, 2),
 (1987, 3),
 (1987, 4),
 (1987, 5),
 (1987, 6),
 (1987, 7),
 (1987, 8),
 (1987, 9),
 (2021, 12)]